### workflow
1. 分析攻击IP、目的IP，确定攻击IP地域（国家、省份/州、城市、县区）、威胁等级、所属运营商、归属人；
2. 综合分析事件描述、证据图片，确定研判结果；
3. 输出研判表单；
    - 排查
    - 封堵
    - 

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Define Tools

In [2]:
import json
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.tools import ToolException

class GetIpInfoInput(BaseModel):
    ip: str = Field(description="查询的IP地址")

def ip_info(ip: str):
    return json.dumps(
        {
            "msg": "操作成功",
            "code": 200,
            "data": {
                "id": None,
                "ip": "115.236.153.174",
                "minip": None,
                "maxip": None,
                "riskTag": None,
                "getTime": None,
                "getStartTime": None,
                "getEndTime": None,
                "riskLevel": None,
                "riskScore": None,
                "continent": None,
                "countryCode": None,
                "country": "中国",
                "province": "浙江省",
                "city": None,
                "district": None,
                "bdLon": None,
                "bdLat": None,
                "wgsLon": None,
                "wgsLat": None,
                "radius": None,
                "isp": "中国电信",
                "owner": None,
                "scene": None,
                "insertTime": None,
                "updateTime": None,
                "adcode": None,
                "timezone": None,
                "accuracy": None,
                "source": "数据中心",
                "asnumber": "58461",
                "areacode": None,
                "zipcode": None,
                "lngwgs": None,
                "latwgs": None
            }
        }
    )

class TrapDetectionInfoInput(BaseModel):
    value: str = Field(description="查询值，可以是IP地址、域名或其他类型的查询项，用于检索与之相关的情报数据。")

# def trap_detection_info(page_num: int, page_size: int, value: str):
def trap_detection_info(value: str):
    return json.dumps(
        {
            "total": 1,
            "rows": [
                {
                    "id": "13f4bfda2dd05df7dc5c9afd97c350c4a5f4387e15b0ccda77b333c5c85d11d3",
                    "iocKey": "115.236.153.174",
                    "ioc1": "115.236.153.174",
                    "ioc2": "53413",
                    "ioc3": "",
                    "iocCategory": "ip",
                    "protocol": "",
                    "maliciousType": "远控木马",
                    "malwareFamily": "Dorkbot",
                    "isApt": False,
                    "campaign": "",
                    "aptGroup": "",
                    "killChain": [
                        ""
                    ],
                    "alertName": "Dorkbot远控木马活动事件",
                    "platform": [
                        "Windows"
                    ],
                    "risk": "high",
                    "confidence": "high",
                    "currentStatus": "unknown",
                    "ttp": "MD5:5676b9d26ab3441d81ef904d60902207Size:2014208FileType:PE32ExePlatform:WINDOWSProcessPath:C:\\Users\\ADMINI~1\\AppData\\Local\\Temp\\svshoct.exePE_Timestamp:2013-01-0918:36:49DNS:TCP:115.236.153.174:1937660.167.153.43:2979HTTP::family:md5:17ec6f6cb75d2afeb72ea848fa78e15epayload:------------------------------------------------------------------------|000102030405060708090A0B0C0D0E0FDump------------------------------------------------------------------------|6e6e1facffffffbffcfffffdffffff779bnn.............w.|5c5f5f97fcc36b3fcb07c314621617977f\\__...k?....b....|6f654e56616da53d3f5d6db17d1efcdafeoeNVam.=?]m.}....|975f9394cbbebfbffd12541cbf570ee9e6._........T..W...|e416e16566e6120ddd15674178800afdbf...ef.....gAx....|c381bf3ff620c8bfef228f8abe22322aae...?....\"...\"2*.|a083030c0bc9bb2e37b2ce6ff4321bed81........7..o.2...|0177f298b4a03f7d6a2f5f635f63ab645f.w....?}j/_c_c.d_|64679f5f6367625f45578058a788081611dg._cgb_EW.X.....|3cdfdce7a06f6fcad9c4e7ddff78f80000<....oo......x...md5:17ec6f6cb75d2afeb72ea848fa78e15efilename:\"-\"size:506880type:PE32ExecutableforMSWindowsfirst_seen:2023-06-1021:05:05processpath:-cmdline:-pe_timestamp:2022-05-0423:27:12family:-susp_family:-malicious_type:-",
                    "description": "",
                    "tags": [
                        "远控木马",
                        "白名单"
                    ],
                    "insertTime": "2023-07-28 18:51:43",
                    "updateTime": None,
                    "createdTime": "2023-09-08 17:20:04",
                    "createdBy": "admin",
                    "updatedTime": None,
                    "updatedBy": None
                }
            ],
            "code": 200,
            "msg": "查询成功"
        }
    )

class OwnDetectionInfoInput(BaseModel):
    value: str = Field(description="查询值可以是IP地址、域名或其他类型的查询项，用于检索与之相关的情报数据。")

def own_detection_info(value: str):
    return json.dumps(
        {
            "total": 1,
            "rows": [
                {
                    "tags": [
                        "远控木马",
                        "白名单"
                    ],
                    "insertTime": "2023-09-08 17:20:04",
                    "updateTime": "2023-09-26 10:23:05",
                    "status": "有效"
                }
            ],
            "code": 200,
            "msg": "查询成功"
        }
    )

def judgment_rules():
    return """1.研判结果为忽略，包括：\
    1）白名单地址：包括各漏洞扫描挖掘团队及集团的扫描器地址以及其他单位提交申请的联通自用IP。\
    2）动态地址：未攻击成功的宽带、基站地址。\
    3）特定IP地址：如跳板机地址和监控地址，这些地址在特定情况下可以忽略。\
2.研判结果为排查，包括：\
    1）联通自用地址：攻击源为联通资产地址，需要经过事件截图分析其行为及特征后，定位到具体归属并下发相关单位或团队排查处置。\
    2）内网IP地址：涉及特定网段的内网IP地址，需要根据归属进行排查。\
    3）家庭宽带、基站攻击：攻击地址为联通家庭宽带、基站地址，需要通过事件截图中的攻击行为及特征进行分析后，协调归属单位进行处置。\
3.研判结果为封堵，包括：\
    1）非联通自用IP：运营商归属联通，使用者非联通，在威胁情报平台被标记为恶意或者存在威胁的IP。\
    2）非联通地址：运营商归属非联通，有扫描及攻击行为的IP。\
    3）境外IP：非联通使用、与联通业务无关的IP。\
    4）对联通有大量扫描和攻击行为的地址：确认过滤各扫描团队的IP之后，对联通自用IP协调归属地单位进行排查处置；非联通自用IP，立即进行封堵。\
    5）多个分子公司上报的同一个地址：多个单位上报同一个地址，经过事件截图分析其行为以及特征之后，过滤扫描团队IP，联通自用IP协调归属地单位进行排查处置，非联通自用IP，立即进行封堵。\
    6）部分家庭宽带及动态IP：非联通地址可封堵，联通地址经过事件截图分析其行为以及特征之后，确认为恶意IP，并且查到相关备案信息非联通自用可封堵。\
"""

def _handle_error(error: ToolException) -> str:
    return (
        "The following errors occurred during tool execution:"
        + error.args[0]
        + "Please try another tool."
    )

get_ip_info = StructuredTool.from_function(
    func=ip_info,
    name="get_ip_information",
    description="获取IP的基本信息接口，专门设计用来查询特定IP地址相关属性的服务。通过这个接口，用户可以获得一个IP地址的多种信息，包括但不限于地理位置、所属的自治系统（ASN）、运营商信息等。",
    args_schema=GetIpInfoInput,
    # return_direct=True,
    handle_tool_error=_handle_error,
)
get_trap_detection_info = StructuredTool.from_function(
    func=trap_detection_info,
    name="get_trap_detection_info",
    args_schema=TrapDetectionInfoInput,
    description="详细的情报信息查询接口，专用于检测网络是否存在失陷情况，此接口可以确定某个IP地址、域名或URL是否与已知的恶意活动相关联。",
    # return_direct=True,
    handle_tool_error=_handle_error,
)
get_own_detection_info = StructuredTool.from_function(
    func=own_detection_info,
    name="get_own_detection_info",
    args_schema=OwnDetectionInfoInput,
    description="简洁明了的情报信息查询接口，专用于检测网络是否存在失陷情况，此接口可以确定某个IP地址、域名或URL是否与已知的恶意活动相关联。",
    handle_tool_error=_handle_error,
)
get_judgment_rules = StructuredTool.from_function(
    func=judgment_rules,
    name="get_judgment_rules",
    description="获取研判规则接口，返回值是详细的研判规则。",
    handle_tool_error=_handle_error,
)



In [3]:
get_ip_info("1")

'{"msg": "\\u64cd\\u4f5c\\u6210\\u529f", "code": 200, "data": {"id": null, "ip": "115.236.153.174", "minip": null, "maxip": null, "riskTag": null, "getTime": null, "getStartTime": null, "getEndTime": null, "riskLevel": null, "riskScore": null, "continent": null, "countryCode": null, "country": "\\u4e2d\\u56fd", "province": "\\u6d59\\u6c5f\\u7701", "city": null, "district": null, "bdLon": null, "bdLat": null, "wgsLon": null, "wgsLat": null, "radius": null, "isp": "\\u4e2d\\u56fd\\u7535\\u4fe1", "owner": null, "scene": null, "insertTime": null, "updateTime": null, "adcode": null, "timezone": null, "accuracy": null, "source": "\\u6570\\u636e\\u4e2d\\u5fc3", "asnumber": "58461", "areacode": null, "zipcode": null, "lngwgs": null, "latwgs": null}}'

## Tool without tool calling

In [5]:
from langchain.tools.render import render_text_description
from langchain_core.prompts import ChatPromptTemplate


rendered_tools = render_text_description(tools=[get_ip_info, get_trap_detection_info, get_own_detection_info, get_judgment_rules])
print(rendered_tools)

system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

get_ip_information: get_ip_information(ip: str) - 获取IP的基本信息接口，专门设计用来查询特定IP地址相关属性的服务。通过这个接口，用户可以获得一个IP地址的多种信息，包括但不限于地理位置、所属的自治系统（ASN）、运营商信息等。
get_trap_detection_info: get_trap_detection_info(value: str) - 详细的情报信息查询接口，专用于检测网络是否存在失陷情况，此接口可以确定某个IP地址、域名或URL是否与已知的恶意活动相关联。
get_own_detection_info: get_own_detection_info(value: str) - 简洁明了的情报信息查询接口，专用于检测网络是否存在失陷情况，此接口可以确定某个IP地址、域名或URL是否与已知的恶意活动相关联。
get_judgment_rules: get_judgment_rules() - 获取研判规则接口，返回值是详细的研判规则。


## Define Execution Agent

In [468]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate

from langgraph.prebuilt import create_agent_executor


In [469]:
def create_agent(llm, tools, system_prompt):
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessagePromptTemplate(prompt=PromptTemplate(template=system_prompt)), 
            MessagesPlaceholder(variable_name='chat_history', optional=True), 
            HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), 
            MessagesPlaceholder(variable_name='agent_scratchpad')
        ]
    )
    agent_runnable = create_openai_tools_agent(llm, tools, prompt)
    # agent_executor = AgentExecutor(agent=agent_runnable, tools=tools)
    agent_executor = create_agent_executor(agent_runnable, tools)
    
    return agent_executor

In [470]:
system_template = f"""你是安全事件研判助手，分析安全事件并基于研判规则得出研判结果。"""

llm = ChatOpenAI(temperature=0)
tools = [get_ip_info, get_trap_detection_info, get_own_detection_info]
agent_executor = create_agent(llm=llm, tools=tools, system_prompt=system_template)

## Running

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

In [471]:
event_desc = '115.236.153.174 Attack 60.78.180.99'
event_details = "SQL注入事件"
attacker_ip = "115.236.153.174"
attacked_ip = "60.78.180.99"

inputs = {
    '事件描述': event_desc,
    '事件详情': event_details,
    '攻击者ip': attacker_ip,
    '被攻击者ip': attacked_ip
}

rules = """
1.研判结果为忽略，包括：\
    1）白名单地址：包括各漏洞扫描挖掘团队及集团的扫描器地址以及其他单位提交申请的联通自用IP。\
    2）动态地址：未攻击成功的宽带、基站地址。\
    3）特定IP地址：如跳板机地址和监控地址，这些地址在特定情况下可以忽略。\
2.研判结果为排查，包括：\
    1）联通自用地址：攻击源为联通资产地址，需要经过事件截图分析其行为及特征后，定位到具体归属并下发相关单位或团队排查处置。\
    2）内网IP地址：涉及特定网段的内网IP地址，需要根据归属进行排查。\
    3）家庭宽带、基站攻击：攻击地址为联通家庭宽带、基站地址，需要通过事件截图中的攻击行为及特征进行分析后，协调归属单位进行处置。\
3.研判结果为封堵，包括：\
    1）非联通自用IP：运营商归属联通，使用者非联通，在威胁情报平台被标记为恶意或者存在威胁的IP。\
    2）非联通地址：运营商归属非联通，有扫描及攻击行为的IP。\
    3）境外IP：非联通使用、与联通业务无关的IP。\
    4）对联通有大量扫描和攻击行为的地址：确认过滤各扫描团队的IP之后，对联通自用IP协调归属地单位进行排查处置；非联通自用IP，立即进行封堵。\
    5）多个分子公司上报的同一个地址：多个单位上报同一个地址，经过事件截图分析其行为以及特征之后，过滤扫描团队IP，联通自用IP协调归属地单位进行排查处置，非联通自用IP，立即进行封堵。\
    6）部分家庭宽带及动态IP：非联通地址可封堵，联通地址经过事件截图分析其行为以及特征之后，确认为恶意IP，并且查到相关备案信息非联通自用可封堵。\
"""

chat_history = [
    ("human", f"你需要遵守以下研判规则：{rules}"),
    ("human", "根据上报的安全事件，一步一步地调用工具以透彻的分析提供的安全事件"),
]

inputs = {"input": json.dumps(inputs), "chat_history": chat_history}

# agent_executor.invoke(
#     input=inputs
# )
for output in agent_executor.stream(inputs):
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'agent_outcome': [OpenAIToolAgentAction(tool='get_ip_information', tool_input={'ip': '115.236.153.174'}, log="\nInvoking: `get_ip_information` with `{'ip': '115.236.153.174'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VzWAbyr89grDRljYvLHh1ZAe', 'function': {'arguments': '{"ip":"115.236.153.174"}', 'name': 'get_ip_information'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 1264, 'total_tokens': 1285}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8827a835-2413-40b9-bd78-4a936b71c319-0')], tool_call_id='call_VzWAbyr89grDRljYvLHh1ZAe')]}

---

Output from node 'action':
---
{'intermediate_steps': [(OpenAIToolAgentAction(tool='get_ip_information', tool_input={'ip': '115.236.153.174'}, log="\nInvoking: `get_ip_information` with `{'ip': '115.236.153.174'}`\n\n\n", message_log=[AI

## TEST

### OpenAI API处理图像函数

In [ ]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8080/v1", 
    api_key = "sk-no-key-required"
)

# import base64

# image_path = '/Users/treasures_y/Desktop/image.png'
# with open(image_path, 'rb') as image_file:
#     image_data = image_file.read()
#     encoded_bytes = base64.b64encode(image_data)
#     decoded_string = encoded_bytes.decode('utf-8')
    
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are ChatGPT, an AI assistant. Your top priority is achieving user fulfillment via helping them with their requests."},
        {
            "role": "user", 
            "content": [
                {
                    "type": "text",
                    "text": "What’s in this image?"
                },
                {
                    "type": "image_url",
                    "image_url": "/Users/treasures_y/Desktop/image.png"
                }
            ]
        },
    ],
    n=1
)
print(completion.choices[0])



In [ ]:
You must first answer the user's request in a straightforward manner. Then describe the task process and show your analysis and model inference results to the user in the first person. 

In [5]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlace